# Copy "Smart Crop Images" into Train

Based on the Train Assignment

Adaptation of `2020-03-28-TestResults.ipynb` notebook

In [4]:
import pandas as pd
import s3fs
import boto3
import json
from datetime import datetime

# Read Train Reference

In [5]:
df_train = pd.read_csv('train_crops.csv')
df_train.head()

,img_id,heading,crop_number,0_missing,1_null,2_obstacle,3_present,4_surface_prob,5_nosidewalk,6_occlusion,count_all,ground_truth
0,510,45,4,0,1,0,0,0,0,0,0,1_null
1,510,45,7,0,0,0,1,0,0,0,1,3_present
2,510,135,3,0,0,0,1,0,0,0,1,3_present
3,510,135,6,0,0,0,1,0,0,0,1,3_present
4,510,135,7,0,0,0,1,0,0,0,1,3_present


In [7]:
df_train['img_heading_folder'] = df_train['img_id'].astype(str) + '_' + df_train['heading'].astype(str)
df_train.head()

,img_id,heading,crop_number,0_missing,1_null,2_obstacle,3_present,4_surface_prob,5_nosidewalk,6_occlusion,count_all,ground_truth,img_heading_folder
0,510,45,4,0,1,0,0,0,0,0,0,1_null,510_45
1,510,45,7,0,0,0,1,0,0,0,1,3_present,510_45
2,510,135,3,0,0,0,1,0,0,0,1,3_present,510_135
3,510,135,6,0,0,0,1,0,0,0,1,3_present,510_135
4,510,135,7,0,0,0,1,0,0,0,1,3_present,510_135


## Create columns for the existing JPG Names and New Names

In [10]:
# New jpg name
df_train['original_image_path'] = 'crops/' + df_train['img_heading_folder'] + '/' + df_train['crop_number'].astype(str) + '.jpg'
df_train['new_image_path'] = 'labeled-train/' +df_train['ground_truth'] + '/'+ df_train['img_heading_folder'] + '_' + df_train['crop_number'].astype(str) + '.jpg'
df_train['image_name'] = df_train['img_heading_folder'] + '_' + df_train['crop_number'].astype(str) + '.jpg'
df_train.head()

,img_id,heading,crop_number,0_missing,1_null,2_obstacle,3_present,4_surface_prob,5_nosidewalk,6_occlusion,count_all,ground_truth,img_heading_folder,original_image_path,new_image_path,image_name
0,510,45,4,0,1,0,0,0,0,0,0,1_null,510_45,crops/510_45/4.jpg,labeled-train/1_null/510_45_4.jpg,510_45_4.jpg
1,510,45,7,0,0,0,1,0,0,0,1,3_present,510_45,crops/510_45/7.jpg,labeled-train/3_present/510_45_7.jpg,510_45_7.jpg
2,510,135,3,0,0,0,1,0,0,0,1,3_present,510_135,crops/510_135/3.jpg,labeled-train/3_present/510_135_3.jpg,510_135_3.jpg
3,510,135,6,0,0,0,1,0,0,0,1,3_present,510_135,crops/510_135/6.jpg,labeled-train/3_present/510_135_6.jpg,510_135_6.jpg
4,510,135,7,0,0,0,1,0,0,0,1,3_present,510_135,crops/510_135/7.jpg,labeled-train/3_present/510_135_7.jpg,510_135_7.jpg


In [11]:
df_train.to_csv('train_crop_filename.csv')

In [22]:
# Grab a sample of images 
#df_train_sample = df_train.iloc[1288:1315]

# Actually Copy files with New Name

In [12]:
# Move files from outer folder to gsv folder
bucket_name = 'gsv-crops2'
s3 = boto3.resource('s3')

start = datetime.now()
print(start)
cannot_move_paths = []
for index, row in df_train.iterrows():
    if index % 500 == 0:
        print(f'Through {index} images at {datetime.now()}')
    # Copy to gsv folder
    old_name = str(row['original_image_path'])
    new_name = str(row['new_image_path'])
    copy_source = {
        'Bucket': bucket_name,
        'Key': old_name
    }
    try:
        s3.meta.client.copy(copy_source, bucket_name, new_name)
    except:
        print(f'Could not move {old_name} at {datetime.now()}')
        cannot_move_paths.append(old_name)
end = datetime.now()
print(end)

2020-03-29 20:39:29.644436
Through 0 images at 2020-03-29 20:39:29.650479
Could not move crops/680_45/1.jpg at 2020-03-29 20:39:30.599370
Could not move crops/680_45/2.jpg at 2020-03-29 20:39:30.610807
Could not move crops/680_45/5.jpg at 2020-03-29 20:39:30.623068
Could not move crops/680_45/7.jpg at 2020-03-29 20:39:30.633706
Could not move crops/680_135/1.jpg at 2020-03-29 20:39:30.643962
Could not move crops/680_135/2.jpg at 2020-03-29 20:39:30.654375
Could not move crops/680_135/3.jpg at 2020-03-29 20:39:30.666506
Could not move crops/680_135/4.jpg at 2020-03-29 20:39:30.677067
Could not move crops/680_135/5.jpg at 2020-03-29 20:39:30.690669
Could not move crops/680_135/6.jpg at 2020-03-29 20:39:30.700136
Could not move crops/680_135/7.jpg at 2020-03-29 20:39:30.710332
Could not move crops/680_225/1.jpg at 2020-03-29 20:39:30.721884
Could not move crops/680_225/2.jpg at 2020-03-29 20:39:30.731819
Could not move crops/680_225/3.jpg at 2020-03-29 20:39:30.741131
Could not move crops

In [13]:
len(cannot_move_paths)

175

In [15]:
with open('train_cannot_move_paths.txt', 'w') as file:
    for path in cannot_move_paths:
        file.write(f'{path}\n')

# Determine Which Panos were not transferred

In [26]:
df_nomove = pd.DataFrame(cannot_move_paths)
df_nomove.columns = ['original_image_path']
df_nomove.head()

,original_image_path
0,crops/680_45/1.jpg
1,crops/680_45/2.jpg
2,crops/680_45/5.jpg
3,crops/680_45/7.jpg
4,crops/680_135/1.jpg


In [27]:
df_nomove_join = df_nomove.merge(df_train, how = 'left', left_on = 'original_image_path', right_on = 'original_image_path')
df_nomove_join.head()

,original_image_path,img_id,heading,crop_number,0_missing,1_null,2_obstacle,3_present,4_surface_prob,5_nosidewalk,6_occlusion,count_all,ground_truth,img_heading_folder,new_image_path,image_name
0,crops/680_45/1.jpg,680,45,1,0,0,0,2,0,0,0,2,3_present,680_45,labeled-train/3_present/680_45_1.jpg,680_45_1.jpg
1,crops/680_45/2.jpg,680,45,2,0,0,0,1,0,0,0,1,3_present,680_45,labeled-train/3_present/680_45_2.jpg,680_45_2.jpg
2,crops/680_45/5.jpg,680,45,5,0,0,0,2,0,0,0,2,3_present,680_45,labeled-train/3_present/680_45_5.jpg,680_45_5.jpg
3,crops/680_45/7.jpg,680,45,7,0,0,0,1,0,0,0,1,3_present,680_45,labeled-train/3_present/680_45_7.jpg,680_45_7.jpg
4,crops/680_135/1.jpg,680,135,1,0,1,0,0,0,0,0,0,1_null,680_135,labeled-train/1_null/680_135_1.jpg,680_135_1.jpg


In [28]:
df_nomove_join.shape

(175, 16)

In [29]:
df_nomove_join['ground_truth'].value_counts()

1_null            90
3_present         50
2_obstacle        22
4_surface_prob    11
0_missing          2
Name: ground_truth, dtype: int64

# Get updated Counts

In [31]:

df_kept = df_train.loc[~df_train['original_image_path'].isin(cannot_move_paths)]
print(df_kept.shape)

(5554, 16)


In [32]:
df_kept['ground_truth'].value_counts()

3_present         2457
1_null            2410
4_surface_prob     401
2_obstacle         154
0_missing          132
Name: ground_truth, dtype: int64